# OpenAI LLM Monitoring

This notebook shows how to use the W&B OpenAI integration to monitor API calls and understand how your projects and teams are leveraging LLMs.

TODO:

- correct openai authentication
- remove devmode + try without it
- when board link first shows up, there is just an empty True brick - no data there if I click it after the first two log calls.

# Step 0: Setup

* authenticate with OpenAI
* login to W&B so you can save and share your work
* install dependencies

In [ ]:
OPENAI_API_KEY = "sk-"

In [ ]:
import openai as openai_access; openai_access.api_key=OPENAI_API_KEY

In [ ]:
import wandb
wandb.login()

In [ ]:
import weave
weave.use_frontend_devmode()
from weave.monitoring import openai, init_monitor

# Step 1: Configure logging

In [ ]:
STREAM_TABLE_NAME = "test72"
WB_PROJECT = "myth"
WB_ENTITY = "stacey"

In [ ]:
OPENAI_MODEL = 'gpt-3.5-turbo'

In [ ]:
# Pass <wandb_team_or_user>/<wandb_project>/<table_name>
m = init_monitor(f"{WB_ENTITY}/{WB_PROJECT}/{STREAM_TABLE_NAME}")

# Do an initial request, otherwise we don't have a type on which to recommend the OpenAI board!
# We need at least 2 requests for the Board to work, otherwise we get divide by zero errors.
# TODO: fix this onboarding issue
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": f"hello world!"}])
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": f"what is 2+2?"}])

Click the link above to go to the Weave UI for the table we're logging to.

From there you can click "OpenAI Monitor Board" to create a Weave Board for this data stream.

In [ ]:
# Monitor ChatCompletion requests
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[
        {"role": "user", "content": f"who won the world series in 2006?"},
    ])
r['choices'][0]['message']['content']

In [ ]:
# To keep track of prompts and parameters, add them to attributes on the logged
# record.

system_prompt = "you always write in bullet points"
prompt = 'solve the following equation step by step: {equation}'
params = {'equation': '4 * (3 - 1)'}
openai.ChatCompletion.create(model=OPENAI_MODEL,
                             messages=[
                                    {"role": "system", "content": system_prompt},
                                    {"role": "user", "content": prompt.format(**params)},
                                ],
                             # you can add additional attributes to the logged record
                             # see the monitor_api notebook for more examples
                             monitor_attributes={
                                 'system_prompt': system_prompt,
                                 'prompt': prompt,
                                 'params': params
                             })

In [ ]:
# Monitor streaming requests
# TODO: we don't get token counts here yet.
from weave.monitoring.openai import message_from_stream
r = openai.ChatCompletion.create(model=OPENAI_MODEL, messages=[
        {"role": "system", "content": "Your are a robot and only speak in robot, like beep bloop bop."},
        {"role": "user", "content": f"Tell me a 50 word story."},
    ], stream=True)
for s in message_from_stream(r):
    print(s, end='')

In [ ]:
import os
from getpass import getpass

if os.getenv("OPENAI_API_KEY") is None:
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")